In [1]:
!pip install flask flask-cors

In [12]:
import threading
from flask import Flask, request, jsonify
from flask_cors import CORS
import os
from query_engine_interface import (
    generate_json_with_openai_few_shot,
    retrieve_context_from_vector_db,
    setup_vector_db
)

# Initialize Flask app
app = Flask(__name__)
CORS(app)

# Ensure the vector database is set up
vector_db_client = setup_vector_db(graph) if graph else None

@app.route("/query", methods=["POST"])
def query_interactions():
    """
    Endpoint to retrieve system interactions for a given query.
    """
    if vector_db_client is None:
        return jsonify({"error": "Vector database not available"}), 503

    data = request.get_json()
    query = data.get("query", "")

    if not query:
        return jsonify({"error": "Query parameter is missing"}), 400

    # Retrieve context
    retrieved_context = retrieve_context_from_vector_db(vector_db_client, query)
    if not retrieved_context:
        return jsonify({"error": "No context found"}), 404

    # Generate JSON response using OpenAI
    json_response = generate_json_with_openai_few_shot(retrieved_context, query)
    if json_response:
        return jsonify({"query": query, "response": json_response}), 200
    else:
        return jsonify({"error": "Failed to generate JSON response"}), 500

# Flask Server Thread
def run_flask():
    """
    Run Flask app on a separate thread to keep the notebook interactive.
    """
    app.run(port=5000, debug=False, use_reloader=False)

# Start Flask in a separate thread
flask_thread = threading.Thread(target=run_flask)
flask_thread.start()

# Display status message
print("Flask server is running on http://127.0.0.1:5000")

Vector database populated with 100 points!
Flask server is running on http://127.0.0.1:5000
 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
